In [1]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go

module_path = os.path.abspath(os.path.join('./../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.multicovariate_models.GroupsModelWrapper import GroupsModelWrapper
from src.multicovariate_models.DataLoader import DataLoader

In [24]:
%%time

base_dir = '../../data/dataframes/'
main_df = pd.read_csv(os.path.join(base_dir, "simulations-step0.01.csv"))
main_df = main_df.drop(columns=["origin_file"]).drop_duplicates()
"""main_df = main_df[['Sensor1', 'Sensor2', 'Sensor3', 'Sensor4', 'J-Apollo', 'J-RN2',
       'J-RN1', 'node_with_leak']].round(3).drop_duplicates()"""

main_df = main_df.drop(DataLoader.SENSOR_COLS, axis=1)

for col_name in DataLoader.SENSOR_DPRESSURE_COLS:
    sen_col_name = col_name[:-10]
    main_df[sen_col_name] = main_df[col_name]

main_df = main_df.drop(DataLoader.SENSOR_DPRESSURE_COLS, axis=1)
display(main_df)

,node_with_leak,leak_amount,Sensor1,Sensor2,Sensor3,Sensor4,J-Apollo,J-RN2,J-RN1
0,Node_275-A,19.5LPS,0.7448,1.5579,0.4998,0.1458,0.0,0.0234,0.0
1,Node_275-A,19.501LPS,0.7449,1.5581,0.4998,0.1458,0.0,0.0234,0.0
2,Node_275-A,19.502LPS,0.7449,1.5582,0.4998,0.1459,0.0,0.0234,0.0
3,Node_275-A,19.503LPS,0.7450,1.5583,0.4999,0.1459,0.0,0.0234,0.0
4,Node_275-A,19.504LPS,0.7450,1.5584,0.4999,0.1459,0.0,0.0234,0.0
...,...,...,...,...,...,...,...,...,...
1009579,Node_Jonctiune-267,7.696LPS,0.2080,0.2893,0.1932,0.0540,0.0,0.0092,0.0
1009580,Node_Jonctiune-267,7.697LPS,0.2080,0.2893,0.1932,0.0540,0.0,0.0092,0.0
1009581,Node_Jonctiune-267,7.698LPS,0.2080,0.2894,0.1933,0.0541,0.0,0.0092,0.0
1009582,Node_Jonctiune-267,7.699LPS,0.2081,0.2894,0.1933,0.0541,0.0,0.0092,0.0


CPU times: user 2.98 s, sys: 376 ms, total: 3.35 s
Wall time: 3.35 s


In [25]:
%%time

# prepare the data for all models
data_loader = DataLoader(main_df)
X_train, X_test, y_train, y_test, enc_node_dict = data_loader.get_random_data_split_by_node()
print(X_train, X_test, y_train, y_test, enc_node_dict)

Train data shape: (907200, 10), test data shape: (100944, 10)
x_train shape: (907200, 7), y_train shape: (907200,)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [19]:
import numpy as np
import sklearn

n_train = X_train.shape[0]
n_test = X_test.shape[0]

X_train_preproc = data_loader.preprocess_dpressure(X_train)
X_test_preproc = data_loader.preprocess_dpressure(X_test)
print(X_train_preproc.sum(axis=1))

for n_neighbours in range(1, 30):
    classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=n_neighbours, p=np.inf)
    classifier.fit(X_train_preproc, y_train)

    y_hat = classifier.predict(X_test_preproc)

    total_diff = np.sum(np.abs(y_test - y_hat) > 0)
    ca = 1.0 - total_diff / n_test

    print(f'n_neighbours: {n_neighbours}, CA: {ca}')

[1. 1. 1. ... 1. 1. 1.]
n_neighbours: 1, CA: 0.4658816425120773
n_neighbours: 2, CA: 0.5167572463768115
n_neighbours: 3, CA: 0.4836956521739131
n_neighbours: 4, CA: 0.509963768115942
n_neighbours: 5, CA: 0.4809782608695652
n_neighbours: 6, CA: 0.5146437198067633
n_neighbours: 7, CA: 0.4788647342995169
n_neighbours: 8, CA: 0.5066425120772946
n_neighbours: 9, CA: 0.48324275362318836
n_neighbours: 10, CA: 0.5040760869565217
n_neighbours: 11, CA: 0.48580917874396135
n_neighbours: 12, CA: 0.5027173913043479
n_neighbours: 13, CA: 0.4788647342995169
n_neighbours: 14, CA: 0.49833937198067635
n_neighbours: 15, CA: 0.4850543478260869
n_neighbours: 16, CA: 0.4965277777777778
n_neighbours: 17, CA: 0.4817330917874396
n_neighbours: 18, CA: 0.49230072463768115
n_neighbours: 19, CA: 0.48339371980676327
n_neighbours: 20, CA: 0.49577294685990336
n_neighbours: 21, CA: 0.48324275362318836
n_neighbours: 22, CA: 0.48852657004830913
n_neighbours: 23, CA: 0.4870169082125604
n_neighbours: 24, CA: 0.48762077294

In [20]:
n_neighbours = 20

classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=n_neighbours, p=np.inf)
classifier.fit(X_train_preproc, y_train)

y_hat = classifier.predict(X_test_preproc)

total_diff = np.sum(np.abs(y_test - y_hat) > 0)
ca = 1.0 - total_diff / n_test

print(f'n_neighbours: {n_neighbours}, CA: {ca}')

n_neighbours: 20, CA: 0.49577294685990336


In [21]:
# how many nodes have 0 response in the network?
row_sum = X_test_preproc.sum(axis=1)
y_test_undetectable_idxs = row_sum < 1e-5
total_undetectable = sum(y_test_undetectable_idxs)
undetectable_perc = total_undetectable / n_test
print(f'total undetectable: {undetectable_perc}, node count: {len(np.unique(y_test))}')

total undetectable: 0.034722222222222224, node count: 144


In [22]:
# Which nodes can I detect accurately?

import json
import wntr
import src.analytics.analytics as analytics

network_fname = '../../data/epanet_networks/Braila_V2022_2_2.inp'
water_network_model = wntr.network.WaterNetworkModel(network_fname)
network_graph = water_network_model.get_graph()

nodeN_to_node_id_map = {nodeN: node_id[5:] for node_id, nodeN in enc_node_dict.items()}

# precompute the distances
node_dist_map = {}
for node_id in network_graph.nodes():
    path_len_vec, node_id_vec = analytics.undir_dist_from_set_vec(network_graph, [node_id])
    node_dist_map[node_id] = {
        'path_len_vec': path_len_vec,
        'node_id_vec': node_id_vec
    }

stats_map = {}
for valN, y_val_int64 in enumerate(y_test):
    y_val_str = nodeN_to_node_id_map[y_val_int64]
    y_hat_val = y_hat[valN]
    is_undetectable = 1 if y_test_undetectable_idxs[valN] else 0
    
    if y_val_str not in stats_map:
        stats_map[y_val_str] = {'n_hits': 0, 'n_misses': 0, 'is_undetectable': is_undetectable, 'misclassify_id_map': {}, 'misclassify_dist_map': {}}
        
    if y_val_int64 == y_hat_val:
        stats_map[y_val_str]['n_hits'] += 1
    if y_val_int64 != y_hat_val:
        stats_map[y_val_str]['n_misses'] += 1
        
        y_hat_val_str = nodeN_to_node_id_map[y_hat_val]
        
        if y_hat_val_str not in stats_map[y_val_str]['misclassify_id_map']:
            stats_map[y_val_str]['misclassify_id_map'][y_hat_val_str] = 0
        stats_map[y_val_str]['misclassify_id_map'][y_hat_val_str] += 1
        
        # get the distance between y and y_hat
        node_dist_props = node_dist_map[y_val_str]
        path_len_vec = node_dist_props['path_len_vec']
        node_id_vec = node_dist_props['node_id_vec']
        
        misclassify_dist = -1
        for nodeN, node_id in enumerate(node_id_vec):
            if node_id == y_hat_val_str:
                misclassify_dist = path_len_vec[nodeN]
        assert misclassify_dist >= 0
        
        misclassify_dist = min(10000, misclassify_dist)
        
        #misclassify_dist = str(path_len_vec[node_id_vec.index(y_hat_val_str)])
        
        if misclassify_dist not in stats_map[y_val_str]['misclassify_dist_map']:
            stats_map[y_val_str]['misclassify_dist_map'][misclassify_dist] = 0
        stats_map[y_val_str]['misclassify_dist_map'][misclassify_dist] += 1

for node_id, node_stats in stats_map.items():
    total_samples = node_stats['n_hits'] + node_stats['n_misses']
    node_stats['hit_perc'] = float(node_stats['n_hits']) / total_samples
    
    misclassify_dist_map = node_stats['misclassify_dist_map']
    total_misclassify_count = 0
    mean_misclassify_dist = 0.0
    max_misclassify_dist = 0.0
    for misclassify_dist, misclassify_count in misclassify_dist_map.items():
        total_misclassify_count += misclassify_count
        mean_misclassify_dist += misclassify_dist*misclassify_count
        max_misclassify_dist = max(max_misclassify_dist, misclassify_dist)
    mean_misclassify_dist /= total_samples
        
    node_stats['mean_misclassify_dist'] = mean_misclassify_dist
    node_stats['max_misclassify_dist'] = max_misclassify_dist
    
with open(os.path.join(base_dir, 'model_stats.json'), 'w') as f_out:
    json_str = json.dumps(stats_map)
    f_out.write(json_str)
    

/home/lstopar/workspace/JSI/projects/naiades/braila-approximate-leakage/src/analytics/analytics.py:38: RuntimeWarning: divide by zero encountered in reciprocal
  np.reciprocal(reached_indicator_vec, out=tmp_n)
